In [1]:
import pandas as pd
import nltk

In [2]:
train_DF=pd.read_csv('train.csv',index_col='id')

In [3]:
train_DF.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
#NAN occurence in Data
#Question2 as 'NAN' in it

#total number of NAN in each Column
#train_DF.isnull().sum()

#Index which have NAN Values
#train_DF.isnull().any(1).nonzero()
#train_DF.iloc[[105780, 201841]]

#Drop The NAN Values
train_DF.reset_index(drop=True,inplace=True)

In [5]:
#Note
#No duplicates in ID

#train_DF['id_duplicate']=train_DF.duplicated(['id'])
#train_DF[train_DF['id_duplicate']==True]

In [6]:
#Note
#duplicates exist in qid1
#qid is the question ID for question 1

#train_DF['qid1_duplicate']=train_DF.duplicated(['qid1'])
#train_DF[train_DF['qid1_duplicate']==True]


In [7]:
#Note
#duplicates exist in qid2
#qid is the question ID for question 2

#train_DF['qid2_duplicate']=train_DF.duplicated(['qid2'])
#train_DF[train_DF['qid2_duplicate']==True]

In [8]:
#Note
# Question can be duplicates between Question1 and question 2
#same question can come up as Question 1 in one example and Question 2 in other
#Total Number of Unique Questions:347345


#qid1=train_DF['qid1']
#qid2=train_DF['qid2']
#qid=qid1+qid2

#Total Number of Unique Questions
#print(len(qid.unique()))

#Total Number of unique Questions in Question 1 + Total Number of Unique Questions in Question 2
#print(len(qid2.unique())+len(qid1.unique()))

#Maximum times a question repeated in data:6
#qid.value_counts()

In [9]:
# Text Analysis


Text Features
- Number of characters
- Number of words
- 'End' Character in Questions(?vs.)
- TFIDf and LSA(30 dimension)
- LDA (check performance- 20 dimensions)
- Question type- What, When , How, Why( more types have to be added)
- Number of Capital Letters, Special Characters


In [10]:
train_DF['question1'] = train_DF['question1'].astype(str)
train_DF['question2'] = train_DF['question2'].astype(str)

In [11]:
#Character Count

In [12]:
def character_count(a,b):
    return len(a), len(b)

train_DF['character_count_in_question1'],train_DF['character_count_in_question2']= zip(*train_DF.apply(lambda row: 
                                                                                                       character_count(row['question1'], row['question2']), axis=1))

In [13]:
#Word Count

In [14]:
def word_count(a,b):
    return len(nltk.tokenize.word_tokenize(a)), len(nltk.tokenize.word_tokenize(b))

train_DF['word_count_in_question1'],train_DF['word_count_in_question2']= zip(*train_DF.apply(lambda row: 
                                                                                                       word_count(row['question1'], row['question2']), axis=1))

In [15]:
#Difference in Word and Character Counts

#count as feature
#Approaches:
#count_difference<range => 0

#train_DF['difference_in_word_count']=abs(train_DF['word_count_in_question1']-train_DF['word_count_in_question2'])
#train_DF['difference_in_char_count']=abs(train_DF['character_count_in_question1']-train_DF['character_count_in_question2'])

#train_DF_diplicates=train_DF[train_DF['is_duplicate']==1]

#difference_in_word_count
#label less than 5 has 1
#train_DF_diplicates['difference_in_word_count'].value_counts()
#train_DF['difference_in_word_count'].value_counts()

#difference_in_character_count
#Label less than 20 as 1
#train_DF_diplicates['difference_in_char_count'].value_counts()

In [19]:
#Last Character Check
def last_character(a,b):
    if a[-1]=='?':
        if b[-1]=='?':
            return 1,1
        else:
            return 1,0
    else:
        if b[-1]=='?':
            return 0,1
        else:
            return 0,0

train_DF['Last_Character_in_question1'],train_DF['last_Character_in_question2']= zip(*train_DF.apply(lambda row: last_character(row['question1'], row['question2']), axis=1))

# LSA with TFIDF

In [23]:
import sklearn

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [36]:
tfidf_vectorizer.fit(train_DF['question1']+train_DF['question2'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [37]:
from six.moves import cPickle
f = open('tfidf_vectorizer.save', 'wb')
cPickle.dump(tfidf_vectorizer, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

In [38]:
tfidf_question_1 = tfidf_vectorizer.transform(train_DF['question1'])
tfidf_question_2 = tfidf_vectorizer.transform(train_DF['question2'])

In [42]:
print("  Actual number of tfidf features: %d" % tfidf_question_1.get_shape()[1])
print("  Actual number of tfidf features: %d" % tfidf_question_2.get_shape()[1])
print("\nPerforming dimensionality reduction using LSA")

  Actual number of tfidf features: 87010
  Actual number of tfidf features: 87010

Performing dimensionality reduction using LSA


In [43]:
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

svd = TruncatedSVD(50)
lsa = make_pipeline(svd, Normalizer(copy=False))
lsa.fit(tfidf_question_1+tfidf_question_2)

Pipeline(memory=None,
     steps=[('truncatedsvd', TruncatedSVD(algorithm='randomized', n_components=50, n_iter=5,
       random_state=None, tol=0.0)), ('normalizer', Normalizer(copy=False, norm='l2'))])

In [51]:
f = open('lsa.save', 'wb')
cPickle.dump(lsa, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

In [44]:
lsa_question1 = lsa.transform(tfidf_question_1)
lsa_question2 = lsa.transform(tfidf_question_2)

# LDA 

In [56]:
from gensim import corpora, models
import gensim

In [59]:
texts=[]
for string in train_DF['question1']+train_DF['question2']:
    texts.append(nltk.tokenize.word_tokenize(string))
    
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [61]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, chunksize=1000,num_topics=25, id2word = dictionary, passes=1)

In [62]:
f = open('lda.save', 'wb')
cPickle.dump(ldamodel, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

In [63]:
ldamodel.save('lda.model')

# LDA with Preprocessing

In [67]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

# list for tokenized documents in loop
preprocessed_texts = []

# loop through document list
for i in train_DF['question1']+train_DF['question2']:

    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    preprocessed_texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
preprocessed_dictionary = corpora.Dictionary(preprocessed_texts)
    
# convert tokenized documents into a document-term matrix
preprocessed_corpus = [preprocessed_dictionary.doc2bow(text) for text in preprocessed_texts]

# generate LDA model
preprocessed_ldamodel = gensim.models.ldamodel.LdaModel(preprocessed_corpus, chunksize=1000, num_topics=50, id2word = preprocessed_dictionary, passes=5)



In [68]:
f = open('preprocessed_lda.save', 'wb')
cPickle.dump(preprocessed_ldamodel, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()

In [69]:
preprocessed_ldamodel.save('preprocessed_lda.model')